In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import shapely
from shapely.geometry import Polygon, LineString, LinearRing, Point, MultiLineString
from shapely.geometry.collection import GeometryCollection
from descartes import PolygonPatch
from itertools import tee
from fibre.optics import snell, R, T

def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)


In [ ]:
print(np.sin(np.radians(280.)))
print(np.cos(np.radians(280.)))

In [ ]:
800*21

In [ ]:
inc_angle = np.radians(10.)

exterior_norm = np.pi
interior_norm = 0.

unit_vector = np.array([np.cos(inc_angle), np.sin(inc_angle)])
unit_line = LineString([unit_vector, np.zeros(2)])

#def rotation2D(angle):
#    return np.array([])
rotated_line = shapely.affinity.rotate(unit_line, exterior_norm, use_radians=True)
rotated_line_interior = shapely.affinity.rotate(unit_line, interior_norm, use_radians=True)
#print(np.degrees(exterior_norm-inc_angle))

def angle_to_line(angle):
    unit_vector = np.array([np.cos(angle), np.sin(angle)])
    unit_line = LineString([np.zeros(2), unit_vector])
    return unit_line

def line_to_angle(line):
    coord1 = line.coords[0]
    coord2 = line.coords[1]
    x = coord2[0]-coord1[0]
    y = coord2[1]-coord1[1]
    angle = np.arctan2(y,x) 
    return angle

#rint(np.degrees(line_to_angle(rotated_line)))
#rint(np.degrees(line_to_angle(rotated_line_interior)))


def get_normal(obj, intersection, plot_line, interior=False):
    side = 0
    min_distance = np.inf
    intersection_side = None
    for p0, p1 in pairwise(obj.exterior.coords):
        line = LineString( [p0,p1])
        ring = LinearRing( [p0,p1, intersection])
        poly = Polygon(ring)
        distance = poly.area/line.length
        if distance < min_distance:
            min_distance = distance
            intersection_side = side
        side += 1
    points = obj.exterior.coords[intersection_side:intersection_side+2]
    #print(points)
    seg_vec = np.array(points[1])-np.array(points[0])    
    seg_vec = seg_vec/np.linalg.norm(seg_vec)
    #print(seg_vec)    
    line = LineString([np.zeros(2), seg_vec])
    if False:
        plot_line = LineString(points)
        print(plot_line)
        centroid = plot_line.centroid
        x, y = plot_line.xy   
        x0 = centroid.coords[0][0]
        y0 = centroid.coords[0][1]
        #rint(x,y)
        plt.plot(x, y, color='tab:blue', linewidth=2, solid_capstyle='round', zorder=1)        
        plt.xlim((x0-1., x0+1.))
        plt.ylim((y0-1., y0+1.))
        
    exterior_line = shapely.affinity.rotate(line, -90., origin=(0.,0.))
    interior_line = shapely.affinity.rotate(line, 90., origin=(0.,0.))
    if False:
        exterior_line_trans = shapely.affinity.translate(exterior_line, xoff=x0, yoff=y0)
        x, y = exterior_line_trans.xy           
        plt.plot(x, y, color='tab:green', linewidth=2, solid_capstyle='round', zorder=1, ls='--')
        
        interior_line_trans = shapely.affinity.translate(interior_line, xoff=x0, yoff=y0)
        x, y = interior_line_trans.xy           
        plt.plot(x, y, color='tab:red', linewidth=2, solid_capstyle='round', zorder=1, ls='--')
    #interior = np.array([-seg_vec[1], seg_vec[0]])        
    #exterior = np.array([seg_vec[1], seg_vec[0]])
    return (np.array(list(interior_line.coords)[1]), np.array(list(exterior_line.coords)[1]))

intersection = np.array([0.5, 0.5])
if sim.objects[0].intersects(Point(intersection)):
    normals = get_normal(sim.objects[0], intersection, True)
    print(normals)    
    exterior_norm_angle = np.arctan2(normals[1][1],normals[1][0])
    interior_norm_angle = np.arctan2(normals[0][1],normals[0][0])
    print(np.degrees(exterior_norm_angle), np.degrees(interior_norm_angle))

In [ ]:
def global_to_local(normal_angle, angle):
    return np.angle(np.exp(1j*(norm_angle+angle)))

print(np.degrees(global_to_local(np.pi, np.pi-np.radians(30.))))

In [ ]:
GRAY = '#999999'

def global_to_local(x_axis, angle):
    return np.angle(np.exp(1j*(angle-x_axis)))

def local_to_global(x_axis, angle):
    return np.angle(np.exp(1j*(x_axis+angle)))

def get_angle_to_norm(norm_vector, angle, print_output=False):
    norm_angle = np.arctan2(norm_vector[1],norm_vector[0])  
    if print_output:
        print("norm_vector: {}".format(norm_vector))
        print("norm angle: {}".format(np.degrees(norm_angle)))
        print("global inc angle: {}".format(np.degrees(angle)))
    local_inc_angle_left1 = global_to_local(norm_angle, angle)
    local_inc_angle_left2 = global_to_local(norm_angle, -angle)
    if print_output:
        print("local inc angle left: {} or {}".format(np.degrees(local_inc_angle_left1), np.degrees(local_inc_angle_left2)))
    #if np.abs(local_inc_angle_left1) < np.abs(local_inc_angle_left2):
    #    local_inc_angle_left = local_inc_angle_left1
    #else:
    #    local_inc_angle_left = local_inc_angle_left2    
    local_inc_angle_left = local_inc_angle_left1
    
    rot_angle = np.angle(np.exp(1j*(angle-np.pi)))
    if print_output:
        print("rot angle: {}".format(np.degrees(rot_angle)))
    local_inc_angle_right1 = global_to_local(norm_angle, rot_angle)
    local_inc_angle_right2 = global_to_local(norm_angle, -rot_angle)
    if print_output:
        print("local inc angle right: {} or {}".format(np.degrees(local_inc_angle_right1), np.degrees(local_inc_angle_right2)))
    #if np.abs(local_inc_angle_right1) < np.abs(local_inc_angle_right2):
    #    local_inc_angle_right = local_inc_angle_right1
    #else:
    #    local_inc_angle_right = local_inc_angle_right2
    local_inc_angle_right = local_inc_angle_right1
    if print_output:
        print("left: {}, right: {}".format(np.degrees(local_inc_angle_left), np.degrees(local_inc_angle_right)))
    if np.abs(local_inc_angle_right) < np.abs(local_inc_angle_left):
        angle_to_norm = local_inc_angle_right
    else:
        angle_to_norm = local_inc_angle_left
    return norm_angle, angle_to_norm
    
class Ray():
    
    def __init__(self, origin, end_point, ref_index=1.0, intensity=1.0, color=GRAY):
        self.origin = origin        
        self.ref_index = ref_index
        self.end_point = end_point
        self.color = color
        self.intensity = intensity
        
    @property
    def prop_angle(self):
        x = self.end_point[0]-self.origin[0]
        y = self.end_point[1]-self.origin[1]        
        angle = np.arctan2(y,x)        
        return angle
        
    def propagate(self, distance, angle):
        end_point_x = distance*np.cos(angle) + self.origin[0]
        end_point_y = distance*np.sin(angle) + self.origin[1]
        end_point = np.array([end_point_x, end_point_y])        
        self.end_point = end_point
        
    def back_propagate(self, distance, angle):
        end_point_x = -distance*np.cos(angle) + self.origin[0]
        end_point_y = -distance*np.sin(angle) + self.origin[1]
        end_point = np.array([end_point_x, end_point_y])        
        self.end_point = end_point
        
    def reverse(self):
        tmp = np.copy(self.origin)
        self.origin = np.copy(self.end_point)
        self.end_point = tmp
        
    def plot(self, ax):
        line = self.get_line()
        x, y = line.xy        
        ax.plot(x, y, color=self.color, linewidth=3, solid_capstyle='round', zorder=1,
                alpha=self.intensity)
        
    def get_line(self):
        return LineString([self.origin, self.end_point])    
    
    def __repr__(self):
        return "{}, {}, {}".format(self.origin, self.end_point, np.degrees(self.prop_angle))

class RaySimulation():
    
    def __init__(self):
        self.rays = []
        self.finished_rays = []
        self.objects = []
        self.ref_index = []
    
    def init_ray(self, prop_angle):
        origin = 0.5*np.array([-np.cos(prop_angle), -np.sin(prop_angle)]) + np.array([0.1, 0.])
        ray = Ray(origin, origin)
        ray.propagate(1.0, prop_angle)
        self.rays.append(ray)
        
    def create_core(self, radius, length):
        polygon = Polygon([(0.1, -radius), (length+0.1, -radius), (length+0.1, radius), (0.1, radius)])
        self.objects.append(polygon)
        self.ref_index.append(1.5)
        
    def create_prism(self, left, right, height, angle):
        polygon = Polygon([(left, -height), (right, -height),
                           (right-np.tan(np.radians(angle))*height*2., height), (left, height)])
        polygon = shapely.geometry.polygon.orient(polygon)
        self.objects.append(polygon)
        self.ref_index.append(1.5)
        
    def init_background(self):
        self.create_core(0.5, 20.)
        #self.create_prism(0.1, 5.1, 0.5, 30.)
        
    def add_cladding(self, inner_radius, outer_radius, length):
        poly_points = []
        poly_points.append((0.1, inner_radius))
        poly_points.append((length+0.1, inner_radius))
        poly_points.append((length+0.1, outer_radius))
        poly_points.append((0.1, outer_radius))
        polygon = shapely.geometry.polygon.orient(Polygon(poly_points))
        self.objects.append(polygon)
        self.ref_index.append(1.48)
        
        poly_points = []
        poly_points.append((0.1, -inner_radius))
        poly_points.append((length+0.1, -inner_radius))
        poly_points.append((length+0.1, -outer_radius))
        poly_points.append((0.1, -outer_radius))
        polygon = shapely.geometry.polygon.orient(Polygon(poly_points))
        self.objects.append(polygon)
        self.ref_index.append(1.48)
        
    def add_oil(self, inner_radius, outer_radius, length, height, period):
        poly_points = []
        poly_points.append((0.1, inner_radius))
        poly_points.append((length+0.1, inner_radius))
        for x in np.linspace(0.1, length+0.1, 200)[::-1]:
            poly_points.append((x, height*np.sin(2*np.pi*x/period)+outer_radius))
        
        #poly_points.append((0.1,0.5))
        polygon = shapely.geometry.polygon.orient(Polygon(poly_points))
        self.objects.append(polygon)
        self.ref_index.append(1.5)
        
        poly_points = []
        poly_points.append((0.1, -inner_radius))
        poly_points.append((length+0.1, -inner_radius))        
        for x in np.linspace(0.1, length+0.1, 200)[::-1]:
            poly_points.append((x, -height*np.sin(2*np.pi*x/period)-outer_radius))
        
        #poly_points.append((0.1,0.5))
        polygon = shapely.geometry.polygon.orient(Polygon(poly_points))
        self.objects.append(polygon)
        self.ref_index.append(1.5)
        
        
    def plot_objects(self, ax):
        for obj in self.objects:
            patch = PolygonPatch(obj, ec='k', fill=False)            
            ax.add_patch(patch)
        
    def plot_rays(self, ax):
        for iray, ray in enumerate(self.finished_rays):
            #print(ray)
            #center = 0.5*(ray.origin + ray.end_point)
            #plt.text(center[0], center[1], "{}-{:.1f}".format(iray, np.degrees(ray.prop_angle)), clip_on=True)
            ray.plot(ax)
        
    def find_intersection(self, obj, ray):
        inter = obj.intersection(ray)
        #print(inter)
        if isinstance(inter, MultiLineString) or isinstance(inter, GeometryCollection):
            inter = inter[0]
        #print(inter)
        first_inter = np.array(inter.coords)        
        return first_inter
        #for side in self.objects[0].
    
    
    
    def find_container(self, coords):
        point = Point(coords)
        for iobj, obj in enumerate(self.objects):
            if obj.contains(point):
                return iobj
        return None
        
    
    def main(self):
        for steps in range(1):
            iray = 0
            while len(self.rays) > 0:
                ray = self.rays[0]
                #print("len(rays): {}, iray: {}, ray: {}".format(len(self.rays), iray, ray))
                if ray.intensity < 0.01:
                    self.rays.remove(ray)
                    self.finished_rays.append(ray)
                    continue
                #if iray > 50:
                #    break
                    
                iray += 1
                for iobj, obj in enumerate(self.objects):
                    line = ray.get_line()
                    #print("intersects: {}, touches: {}".format(line.intersects(obj),line.touches(obj)))
                    if line.intersects(obj) and not line.touches(obj):
                        if iray == 8:
                            print_stuff = True
                        else:
                            print_stuff = False
                        intersections = self.find_intersection(obj, line)
                        if print_stuff:
                            print("intersections: {}".format(intersections))                        
                        if np.all(np.isclose(intersections[0,:], ray.origin)):
                            if intersections.shape[0] == 1:
                                continue
                            else:
                                intersection = intersections[1,:]
                        else:
                            intersection = intersections[0,:]
                        ray.end_point = intersection
                        #print("intersection: {}".format(intersection))
                        
                        if np.all(np.isclose(intersection, ray.origin)):
                            continue
                        
                        normals = get_normal(obj, intersection, print_stuff)                        
                        if print_stuff:
                            print("inc angle global: {}".format(np.degrees(ray.prop_angle)))
                        
                        if print_stuff:
                            print("normals[0]: {}".format(normals[0]))
                            print("normals[1]: {}".format(normals[1]))
                        norm_angle_int, local_angle_to_int_norm = get_angle_to_norm(normals[0],
                                                                                    ray.prop_angle,
                                                                                    print_output=False)
                        norm_angle_ext, local_angle_to_ext_norm = get_angle_to_norm(normals[1],
                                                                                    ray.prop_angle,
                                                                                    print_output=False)
                        
                        if np.isclose(norm_angle_ext,0.):
                            continue                        
                        
                        inside_point = intersection + normals[0]*1e-2
                        outside_point = intersection + normals[1]*1e-2
                        
                        line_to_outside = LineString((ray.origin, outside_point))
                        line_to_inside = LineString((ray.origin, inside_point))
                        
                        if line_to_outside.length < line_to_inside.length:
                            local_angle = local_angle_to_ext_norm
                            normal_angle = norm_angle_ext
                            conj_normal_angle = norm_angle_int
                            if print_stuff:
                                print("local angle relative to ext norm: {}".format(np.degrees(normal_angle)))
                        else:
                            local_angle = local_angle_to_int_norm
                            normal_angle = norm_angle_int
                            conj_normal_angle = norm_angle_ext
                            if print_stuff:
                                print("local angle relative to int norm: {}".format(np.degrees(normal_angle)))
                        #if iray == 5:
                        #    local_angle = local_angle_to_int_norm
                        #    normal_angle = interior_norm_angle
                        #    print("local angle relative to int norm")
                        if print_stuff:
                            print("global inc angle: {}".format(np.degrees(ray.prop_angle)))
                            print("local inc angle: {}".format(np.degrees(local_angle)))                        
                        
                        refl_angle = -local_angle
                        if print_stuff:
                            print("refl angle local: {}".format(np.degrees(refl_angle)))
                        refl_angle_global = local_to_global(normal_angle, refl_angle)
                        if print_stuff:
                            print("refl angle global: {}".format(np.degrees(refl_angle_global)))
                        #refl_angle_global = np.angle(np.exp(1j*(refl_angle_global+np.pi)))
                        #if print_stuff:
                        #    print("refl angle global rotated: {}".format(np.degrees(refl_angle_global)))
                        
                                                
                        
                        outside_obj = self.find_container(outside_point)
                        inside_obj = self.find_container(inside_point)
                        
                        #print("this object: {}".format(iobj))
                        #print("outside obj: {}".format(outside_obj))
                        #print("inside obj: {}".format(inside_obj))
                        
                        line = ray.get_line()
                        if obj.contains(line.centroid):
                            is_inside = True
                        else:
                            is_inside = False
                        
                        
                        if outside_obj is None:
                            outside_n = 1.0
                        else:
                            outside_n = self.ref_index[outside_obj]
                            
                        if inside_obj is None:
                            inside_n = 1.0
                        else:
                            inside_n = self.ref_index[inside_obj]
                        #print("is inside: {}".format(is_inside))
                        if is_inside:
                            n2 = outside_n                            
                            n1 = inside_n
                        else:
                            n1 = outside_n                            
                            n2 = inside_n
                            
                        if outside_n == 1.0:
                            dist = 5.
                        else:
                            dist = 5*np.sqrt(2.)
                        
                        refl_intensity = R(n1, local_angle, n2)
                        refl = Ray(intersection, intersection, color='tab:red', intensity=refl_intensity*ray.intensity)
                        refl.propagate(dist, refl_angle_global)
                        #refl.reverse()
                        #print("n1: {}, n2: {}".format(n1, n2))
                        
                        
                        local_trans_angle = snell(n1, local_angle, n2)
                        trans_intensity = T(n1, local_angle, n2)
                        if print_stuff:
                            print("trans angle local: {}".format(np.degrees(local_trans_angle)))
                        global_trans_angle = local_to_global(conj_normal_angle, local_trans_angle)
                        if print_stuff:
                            print("trans angle global: {}".format(np.degrees(global_trans_angle)))
                            print("trans intensity: {}".format(trans_intensity))
                        if print_stuff:
                            trans = Ray(intersection, intersection, color='tab:blue', intensity=trans_intensity*ray.intensity)
                        else:
                            trans = Ray(intersection, intersection, color='tab:green', intensity=trans_intensity*ray.intensity)
                        trans.propagate(dist, global_trans_angle)
                        #print("transmission: {}".format(refl))
                        #self.rays.append(refl)                        
                        self.rays.append(refl)                        
                        self.rays.append(trans)
                        break                
                self.rays.remove(ray)
                self.finished_rays.append(ray)
                
                        
                        
                

    
fig, ax = plt.subplots(1,1, figsize=(20,8))
aspect_ratio = 4./10.
#ray.plot(ax)
sim = RaySimulation()
sim.init_ray(np.radians(10.))
#sim.rays[0].propagate(np.sqrt(2), )
sim.init_background()
#inner_radius, outer_radius, length, height, period
sim.add_cladding(0.5, 0.6, 20.)
sim.add_oil(0.6, 0.9, 20., 0.1, 2)

#sim.plot_background(ax)
sim.plot_objects(ax)
#sim.find_intersection(sim.rays[0])
sim.main()
sim.plot_rays(ax)
#sim.rays[0].plot(ax)
#sim.rays[1].plot(ax)
#sim.rays[2].plot(ax)
plt.gca().axis("equal")
#xlims = np.array([-0.1, 0.1]) + 1.17828136
#ylims = np.array([-0.1, 0.1]) + 0.7473554 
#plt.xlim(xlims)
#plt.ylim(ylims)
#ls = LineString(normal)
#x, y = ls.xy
#ax.plot(x, y, color=GRAY, linewidth=3, solid_capstyle='round', zorder=1)

In [ ]:
p1 = sim.objects[0].exterior.coords[1]
p2 = sim.objects[0].exterior.coords[2]
print(p1, p2)
delta_y = p2[1]-p1[1]
print(delta_y)

delta_x = p2[0]-p1[0]
print(delta_x)
print(np.degrees(np.arctan2(delta_x,delta_y)))

In [ ]:
1600*0.5*1.5

In [ ]:
#inc_angle = -(np.pi-np.radians(14.150839253775283))
inc_angle = np.radians(14.150839253775283)
norm_angle = np.radians(-96.95217651978207)
#norm_angle = np.radians(-90)
np.degrees(norm_angle+inc_angle)

In [ ]:
sim.objects[0].exterior.xy


In [ ]:
np.degrees(np.arctan2(-0.9921176262836736, 0.1253100778758405))

In [ ]:
inc_angle = np.radians(14.)
norm_angle1 = np.radians(84.)
norm_angle2 = np.radians(-96.)
local = inc_angle+norm_angle2
print(np.degrees(local))
local_refl = -local
#local_refl = local
print(np.degrees(local_refl))
global_refl = norm_angle2+local_refl
print(np.degrees(global_refl))

In [ ]:
fig, ax = plt.subplots(1,1)
inc_angle = np.radians(10.)

def plot_angle(angle):
    ray_x = np.array([-np.cos(angle), np.cos(angle)])
    ray_y = np.array([-np.sin(angle), np.sin(angle)])
    plt.plot(ray_x, ray_y)

plot_angle(inc_angle)


global_ref_frame_angle = 0.
local_ref_frame_angle = np.radians(80.)
conj_local_ref_frame_angle = local_ref_frame_angle-np.pi
print("External Ref Frame Angle: {}".format(np.degrees(local_ref_frame_angle)))
print("Internal Ref Frame Angle: {}".format(np.degrees(conj_local_ref_frame_angle)))

plot_angle(local_ref_frame_angle)
plot_angle(local_ref_frame_angle+np.pi*0.5)

def global_to_local(x_axis, angle):
    return np.angle(np.exp(1j*(x_axis+angle)))

if False:
    local_inc_angle_left = global_to_local(local_ref_frame_angle, np.pi-inc_angle)
    local_inc_angle_right = global_to_local(local_ref_frame_angle, -inc_angle)
    print("Local Inc Angle Left: {}".format(np.degrees(local_inc_angle_left)))
    print("Local Inc Angle Right: {}".format(np.degrees(local_inc_angle_right)))
    if np.abs(local_inc_angle_right) < np.abs(local_inc_angle_left):
        print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_right)))
    else:
        print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_left)))

if True:
    local_inc_angle_right = global_to_local(conj_local_ref_frame_angle, -inc_angle)
    local_inc_angle_left = global_to_local(conj_local_ref_frame_angle, inc_angle)
    print("Local Inc Angle Left: {}".format(np.degrees(local_inc_angle_left)))
    print("Local Inc Angle Right: {}".format(np.degrees(local_inc_angle_right)))

    if np.abs(local_inc_angle_right) < np.abs(local_inc_angle_left):
        print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_right)))
        local_inc_angle = local_inc_angle_right
    else:
        print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_left)))
        local_inc_angle = local_inc_angle_left
if False:
    #local_inc_angle = global_to_local(conj_local_ref_frame_angle, local_inc_angle)
    print("Local Inc Angle2: {}".format(np.degrees(local_inc_angle)))


    refl_angle_local = -local_inc_angle
    print("Local Refl Angle: {}".format(np.degrees(refl_angle_local)))

    #refl_angle_global = conj_local_ref_frame_angle+refl_angle_local
    #print(np.degrees(refl_angle_global))
    #plot_angle(refl_angle_global)


In [ ]:

        
    
    
norm_angles = np.linspace(-np.pi, np.pi, 100)
inc_angle_global = 45.

inc_angle_left_all = np.zeros(norm_angles.shape)
inc_angle_right_all = np.zeros(norm_angles.shape)

refl_local_all_1 = np.zeros(norm_angles.shape)
refl_local_all_2 = np.zeros(norm_angles.shape)

refl_global_all_1 = np.zeros(norm_angles.shape)
refl_global_all_2 = np.zeros(norm_angles.shape)
for ii, norm_angle in enumerate(norm_angles):    
    local_ref_frame_angle = norm_angle
    conj_local_ref_frame_angle = local_ref_frame_angle-np.pi
    #print("External Ref Frame Angle: {}".format(np.degrees(local_ref_frame_angle)))
    #print("Internal Ref Frame Angle: {}".format(np.degrees(conj_local_ref_frame_angle)))

    #plot_angle(local_ref_frame_angle)
    #plot_angle(local_ref_frame_angle+np.pi*0.5)

    if True:
        local_inc_angle_left = global_to_local(local_ref_frame_angle, inc_angle-np.pi)
        local_inc_angle_right = global_to_local(local_ref_frame_angle, inc_angle)
        #print("Local Inc Angle Left: {}".format(np.degrees(local_inc_angle_left)))
        #print("Local Inc Angle Right: {}".format(np.degrees(local_inc_angle_right)))
        if np.abs(local_inc_angle_right) < np.abs(local_inc_angle_left):            
            local_inc_1 = local_inc_angle_right
            #print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_right)))
        else:
            local_inc_1 = local_inc_angle_left
            #print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_left)))

    if False:
        local_inc_angle_right = global_to_local(conj_local_ref_frame_angle, -inc_angle)
        local_inc_angle_left = global_to_local(conj_local_ref_frame_angle, np.pi-inc_angle)
        #print("Local Inc Angle Left: {}".format(np.degrees(local_inc_angle_left)))
        #print("Local Inc Angle Right: {}".format(np.degrees(local_inc_angle_right)))
        if np.abs(local_inc_angle_right) < np.abs(local_inc_angle_left):
            #print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_right)))
            local_inc_2 = local_inc_angle_right
        else:
            #print("Angle to Ext Norm: {}".format(np.degrees(local_inc_angle_left)))
            local_inc_2 = local_inc_angle_left
            #local_inc_angle = local_inc_angle_left
    
    inc_angle_left_all[ii] = local_inc_angle_left
    inc_angle_right_all[ii] = local_inc_angle_right
    
    refl_local1 = -local_inc_1
    refl_local2 = -local_inc_2
    
    refl_local_all_1[ii] = refl_local1
    refl_local_all_2[ii] = refl_local2
    
    
    #inc_angle_local_1[ii] = local_inc_1
    #refl_global1 = local_to_global(local_ref_frame_angle , refl_local1)
    #refl_global2 = local_to_global(conj_local_ref_frame_angle ,refl_local2)

    #refl_global_all_1[ii] = refl_global1
    #refl_global_all_2[ii] = refl_global2
    
#plt.plot(np.degrees(inc_angle_left_all), np.degrees(refl_local_all_1))
#plt.plot(np.degrees(inc_angle_right_all), np.degrees(refl_local_all_1))
plt.plot(np.degrees(inc_angle_left_all), np.degrees(refl_local_all_1))
plt.plot(np.degrees(inc_angle_right_all), np.degrees(refl_local_all_1))
plt.gca().axis("equal")
#print(np.degrees(np.min(inc_angle_local_1)))
#print(np.degrees(np.max(inc_angle_local_1)))

In [ ]:
def global_to_local(x_axis, angle):
    return np.angle(np.exp(1j*(angle-x_axis)))

def local_to_global(x_axis, angle):
    return np.angle(np.exp(1j*(x_axis+angle)))

inc_angle = np.radians(20.)
global1 = np.radians(0.)
global2 = np.radians(10.)
global3 = np.radians(90.)
global4 = np.radians(-180.)
global5 = np.radians(-90.)
print("normal angles angles:")
print(np.degrees(global1))
print(np.degrees(global2))
print(np.degrees(global3))
print(np.degrees(global4))
print(np.degrees(global5))
print("##############")
print("local inc angles:")
local1 = global_to_local(global1, inc_angle)
local2 = global_to_local(global2, inc_angle)
local3 = global_to_local(global3, inc_angle)
local4 = global_to_local(global4, inc_angle)
local5 = global_to_local(global5, inc_angle)
print(np.degrees(local1))
print(np.degrees(local2))
print(np.degrees(local3))
print(np.degrees(local4))
print(np.degrees(local5))
print("##############")
print("global inc angles recalc:")
recalc_global_1 = local_to_global(global1, local1) 
recalc_global_2 = local_to_global(global2, local2)
recalc_global_3 = local_to_global(global3, local3)
recalc_global_4 = local_to_global(global4, local4)
recalc_global_5 = local_to_global(global5, local5)
print(np.degrees(recalc_global_1))
print(np.degrees(recalc_global_2))
print(np.degrees(recalc_global_3))
print(np.degrees(recalc_global_4))
print(np.degrees(recalc_global_5))

print("##############")
print("global refl angles:")
refl_global_1 = local_to_global(global1, -local1) 
refl_global_2 = local_to_global(global2, -local2)
refl_global_3 = local_to_global(global3, -local3)
refl_global_4 = local_to_global(global4, -local4)
refl_global_5 = local_to_global(global5, -local5)
print(np.degrees(refl_global_1))
print(np.degrees(refl_global_2))
print(np.degrees(refl_global_3))
print(np.degrees(refl_global_4))
print(np.degrees(refl_global_5))

